In [12]:
import tensorflow as tf

# Get the GPU device name. 
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import os
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


Found GPU at: /device:GPU:0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [13]:
embed_pick=pd.read_pickle("/content/drive/My Drive/title_embeddings.pkl") #load the embeddings of the json page title
brand_dict=pd.read_pickle("/content/drive/My Drive/brand_dict.pickle") #load the embeddings of the json page title
model_dict=pd.read_pickle("/content/drive/My Drive/model_dict.pkl") #load the embeddings of the json page title

In [15]:
import pandas as pd
non_match_pairs=set()
non_match_pairs_by_brand=dict()
match_pairs=set()
t_data=pd.read_csv("/content/drive/My Drive/ground_truth.csv")
for index, row in t_data.iterrows():
    if row['label']==1 and (brand_dict[row['left_spec_id']]!=brand_dict[row['right_spec_id']]):
        print("Mismatch (which we fix)")
        print(row['left_spec_id'])
        print(row['right_spec_id'])
        t_data.loc[index,'label']=0#We fix it as we think it is an error from the ground truth...!
    if row['label']==1:
        match_pairs.add(row['left_spec_id']+"SEPARATOR"+row['right_spec_id'])
    elif row['label']==0 and (brand_dict[row['left_spec_id']]==brand_dict[row['right_spec_id']]):
        non_match_pairs.add(row['left_spec_id']+"SEPARATOR"+row['right_spec_id'])
        if not brand_dict[row['left_spec_id']] in non_match_pairs_by_brand:
            non_match_pairs_by_brand[brand_dict[row['left_spec_id']]]=1
        else:
            non_match_pairs_by_brand[brand_dict[row['left_spec_id']]]+=1
match_pairs=list(match_pairs)
non_match_pairs=list(non_match_pairs)
print(len(match_pairs))
print(len(non_match_pairs))
print(non_match_pairs_by_brand)

Mismatch (which we fix)
www.ebay.com/21318
www.ohc24.ch/797
1073
9010
{'elo': 329, 'dell': 3013, 'nec': 340, 'philips': 1478, 'lg': 8, 'samsung': 381, 'apple': 143, 'rockwell': 374, 'AOC': 198, 'hp': 205, 'benq': 56, 'asus': 2239, 'lenovo': 6, '3M': 228, 'acer': 12}


In [16]:
train_data=pd.DataFrame(columns=['Left_Title', 'Right_Title', 'Label'])#create a new dataframe

In [17]:
import random
from difflib import SequenceMatcher
random.shuffle(non_match_pairs)
random.shuffle(match_pairs)
for_non_train=list()
unexpected_count=0
##repace the left and right page titles with their bert embeddings
for row in match_pairs:
    left=embed_pick.loc[embed_pick['spec'] == row.split("SEPARATOR")[0].replace("/","//")].embeddings
    right=embed_pick.loc[embed_pick['spec'] == row.split("SEPARATOR")[1].replace("/","//")].embeddings
    train_data=train_data.append({'Left_Title': list(left)[0], 'Right_Title': list(right)[0], 'Label': 1 }, ignore_index=True)
    if model_dict[row.split("SEPARATOR")[0]]!=model_dict[row.split("SEPARATOR")[1]] and SequenceMatcher(None, model_dict[row.split("SEPARATOR")[0]], model_dict[row.split("SEPARATOR")[1]]).ratio() < 0.5:
        print(model_dict[row.split("SEPARATOR")[0]]+"----"+model_dict[row.split("SEPARATOR")[1]]+"-----"+str(SequenceMatcher(None, model_dict[row.split("SEPARATOR")[0]], model_dict[row.split("SEPARATOR")[1]]).ratio()))
        unexpected_count+=1
for row in non_match_pairs:
    left=embed_pick.loc[embed_pick['spec'] == row.split("SEPARATOR")[0].replace("/","//")].embeddings
    right=embed_pick.loc[embed_pick['spec'] == row.split("SEPARATOR")[1].replace("/","//")].embeddings
    if model_dict[row.split("SEPARATOR")[0]]==model_dict[row.split("SEPARATOR")[1]] or SequenceMatcher(None, model_dict[row.split("SEPARATOR")[0]], model_dict[row.split("SEPARATOR")[1]]).ratio() > 0.5:
        train_data=train_data.append({'Left_Title': list(left)[0], 'Right_Title': list(right)[0], 'Label': 0 }, ignore_index=True)
    #for_non_train.append({'Left_Title': list(left)[0], 'Right_Title': list(right)[0], 'Label': 0 })
train_data = train_data.sample(frac=1).reset_index(drop=True)
print(str(unexpected_count))


e178fp----lcd8205-----0.15384615384615385
1


In [18]:
train_data

,Left_Title,Right_Title,Label
0,"[-0.14258593, 1.0876138, -0.7027058, 0.0245024...","[0.385037, 0.8773844, -0.7771925, -0.41962168,...",1
1,"[-0.14258593, 1.0876138, -0.7027058, 0.0245024...","[-0.2754211, 1.2161496, -0.81139195, -0.673738...",0
2,"[-0.045994535, 0.5719848, -0.054123655, 0.0637...","[0.22393546, 0.7994133, -0.54887587, 0.1251741...",0
3,"[-0.0017912617, 0.50674677, 0.2934466, 0.20736...","[-0.19513786, 0.4408024, -0.6041947, -0.052804...",1
4,"[-0.14807041, 0.71383655, 0.05326492, 0.045866...","[-0.14807041, 0.71383655, 0.05326492, 0.045866...",1
...,...,...,...
3760,"[0.15827227, 0.89096004, 0.04024145, -0.352201...","[0.22063379, 0.77158785, -0.2285516, -0.348369...",0
3761,"[0.041627787, 1.0514015, 0.7180133, -0.1084840...","[-0.10499406, 0.96824884, 0.890144, -0.1647403...",0
3762,"[0.015242952, 0.5777541, 0.18430081, -0.052737...","[0.12678085, 1.2389818, 1.1932448, 0.30212384,...",1
3763,"[0.3090158, 0.69809794, 0.10772467, -0.2525184...","[0.015242952, 0.5777541, 0.18430081, -0.052737...",0


In [19]:
##find the difference between the embeddings
train_data['difference'] = [abs(x - y) for x,y in zip(train_data['Left_Title'], train_data['Right_Title'])]


Resultant vector : Difference between left spec and right spec data



In [ ]:
train_data.head()


# Simple KNN

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier



In [21]:
labels= train_data["Label"].to_list()
data_df = train_data["difference"].to_list()

In [22]:
len(data_df[0])

768

In [23]:
##split the data set 
Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(data_df, labels, test_size=0.33, random_state=1000)
for item in for_non_train:
    Emb_test.append(list(abs(x - y) for x,y in zip(item['Left_Title'], item['Right_Title'])))
    Emb_test_label.append(0)

In [24]:
# k = 3
#train
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(Emb_train, Emb_train_label)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

Save File

In [25]:
import pickle 

In [26]:


# Its important to use binary mode 
knnPickle = open('/content/drive/My Drive//knn', 'wb') 

# source, destination 
pickle.dump(classifier, knnPickle)                      




In [27]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(max_iter=100).fit(Emb_train, Emb_train_label)

# Its important to use binary mode 
hgbPickle = open('/content/drive/My Drive//hgb', 'wb') 

# source, destination 
pickle.dump(hgb, hgbPickle)   

In [28]:
# load the model from disk
loaded_model = pickle.load(open('/content/drive/My Drive/knn', 'rb'))

In [29]:
#print(Emb_test[0])
#print(Emb_test[2000])

In [30]:
pred=loaded_model.predict(Emb_test)

In [31]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print('Accuracy is: ',accuracy_score(Emb_test_label, pred))
con_mat = confusion_matrix(Emb_test_label, pred, [0, 1])

print(classification_report(Emb_test_label,pred,target_names=["class_0", "class_1"]))

Accuracy is:  0.7393403057119872
              precision    recall  f1-score   support

     class_0       0.79      0.87      0.82       873
     class_1       0.58      0.44      0.50       370

    accuracy                           0.74      1243
   macro avg       0.68      0.65      0.66      1243
weighted avg       0.72      0.74      0.73      1243



In [32]:
pred=hgb.predict(Emb_test)
print('Accuracy is: ',accuracy_score(Emb_test_label, pred))
con_mat = confusion_matrix(Emb_test_label, pred, [0, 1])

print(classification_report(Emb_test_label,pred,target_names=["class_0", "class_1"]))

Accuracy is:  0.7602574416733708
              precision    recall  f1-score   support

     class_0       0.76      0.96      0.85       873
     class_1       0.76      0.28      0.41       370

    accuracy                           0.76      1243
   macro avg       0.76      0.62      0.63      1243
weighted avg       0.76      0.76      0.72      1243



In [37]:
#Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(equal_data, equal_labels, test_size=0.50) #splitting on 50% data
Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(data_df, labels, test_size=0.33, random_state=1000)
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(Emb_train, Emb_train_label)

pred=classifier.predict(Emb_test)
print('accuracy is: ',accuracy_score(Emb_test_label, pred))
print(classification_report(Emb_test_label,pred,target_names=["class_0", "class_1"]))

accuracy is:  0.7425583266291231
              precision    recall  f1-score   support

     class_0       0.80      0.84      0.82       873
     class_1       0.57      0.52      0.55       370

    accuracy                           0.74      1243
   macro avg       0.69      0.68      0.68      1243
weighted avg       0.74      0.74      0.74      1243



In [39]:
#Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(equal_data, equal_labels, test_size=0.30) #splitting on 50% data
Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(data_df, labels, test_size=0.33, random_state=1000)
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(Emb_train, Emb_train_label)

pred=classifier.predict(Emb_test)
print('accuracy is: ',accuracy_score(Emb_test_label, pred))
print(classification_report(Emb_test_label,pred,target_names=["class_0", "class_1"]))

accuracy is:  0.7393403057119872
              precision    recall  f1-score   support

     class_0       0.79      0.87      0.82       873
     class_1       0.58      0.44      0.50       370

    accuracy                           0.74      1243
   macro avg       0.68      0.65      0.66      1243
weighted avg       0.72      0.74      0.73      1243



#Try different splits

In [42]:
#Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(equal_data, equal_labels, test_size=0.50) #splitting on 50% data
Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(data_df, labels, test_size=0.50, random_state=1000)
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(Emb_train, Emb_train_label)

pred=classifier.predict(Emb_test)
print('accuracy is: ',accuracy_score(Emb_test_label, pred))
print(classification_report(Emb_test_label,pred,target_names=["class_0", "class_1"]))

accuracy is:  0.7360594795539034
              precision    recall  f1-score   support

     class_0       0.81      0.83      0.82      1332
     class_1       0.55      0.52      0.54       551

    accuracy                           0.74      1883
   macro avg       0.68      0.67      0.68      1883
weighted avg       0.73      0.74      0.73      1883



In [44]:
#Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(equal_data, equal_labels, test_size=0.50) #splitting on 50% data
Emb_train, Emb_test, Emb_train_label, Emb_test_label = train_test_split(data_df, labels, test_size=0.50, random_state=1000)
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(Emb_train, Emb_train_label)

pred=classifier.predict(Emb_test)
print('accuracy is: ',accuracy_score(Emb_test_label, pred))
print(classification_report(Emb_test_label,pred,target_names=["class_0", "class_1"]))

accuracy is:  0.7525225703664365
              precision    recall  f1-score   support

     class_0       0.79      0.88      0.83      1332
     class_1       0.60      0.45      0.52       551

    accuracy                           0.75      1883
   macro avg       0.70      0.66      0.68      1883
weighted avg       0.74      0.75      0.74      1883



In [45]:
inference_data=pd.read_csv("/content/drive/My Drive/pairs_to_evaluate.csv")

In [ ]:
output=pd.DataFrame(columns=['left_spec_id', 'right_spec_id'])#create a new dataframe
print("left_spec_id","right_spec_id")
for index, row in inference_data.iterrows():
    a=row["left"]
    b=row["right"]
    left=embed_pick.loc[embed_pick['spec'] == a.replace("/","//")].embeddings
    right=embed_pick.loc[embed_pick['spec'] == b.replace("/","//")].embeddings
    difference=[abs(x - y) for x,y in zip(left, right)]
    pred=hgb.predict(difference)
    if pred==1:
        output.append({'left_spec_id': a.replace("/","//"), 'right_spec_id': b.replace("/","//")}, ignore_index=True)
        #print(a.replace("/","//")+","+b.replace("/","//"))
    if index%100==0:
        print(index)
output.to_csv("/content/output.csv", sep=',',encoding='utf-8', index=False)
